In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Linear Reg").getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
lr = LinearRegression(featuresCol= "features",labelCol= "label", predictionCol= "prediction")

In [11]:
lrModel = lr.fit(training)

In [12]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [13]:
lrModel.intercept

0.14228558260358093

In [14]:
training_summary = lrModel.summary

In [15]:
training_summary.rootMeanSquaredError

10.16309157133015

In [16]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [21]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [22]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
|-26.736207182601724|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
|-17.065399625876015|(10,[0,1,2,3,4,5,...|
+----------

In [23]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                339|
|   mean|0.05601365296907612|
| stddev| 10.436389561763665|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [24]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                162|
|   mean| 0.6772387133529252|
| stddev| 10.084334235770129|
|    min|-21.432387764165806|
|    max| 26.903524792043335|
+-------+-------------------+



In [25]:
correct_model = lr.fit(train_data)

In [26]:
test_results = correct_model.evaluate(test_data)

In [29]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-22.002242719283515|
|-19.529479033144717|
|-21.566589420155758|
| -18.50177756328826|
|-17.076298680374713|
|-16.749480488516646|
| -13.73870193025044|
|-21.191897447345085|
|-15.646990573570907|
|-18.952121332445117|
|-17.977134688692114|
|-11.848338840025372|
|-18.439336302033468|
| -13.67392057168627|
|  -16.9579995875138|
| -8.254683523096599|
| -8.210123248856288|
|  -15.7397967380843|
| -11.08215853263639|
|-11.379849358235512|
+-------------------+
only showing top 20 rows



In [30]:
test_results.rootMeanSquaredError

10.311780386483884

In [31]:
unlabeled_data = test_data.select("features")

In [32]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [33]:
predictions = correct_model.transform(unlabeled_data)

In [34]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| 0.5698549551177094|
|(10,[0,1,2,3,4,5,...|-0.6825982258139531|
|(10,[0,1,2,3,4,5,...|  1.682028645882332|
|(10,[0,1,2,3,4,5,...|-1.3712134747801459|
|(10,[0,1,2,3,4,5,...| 0.3572018467696217|
|(10,[0,1,2,3,4,5,...| 0.5981311372395322|
|(10,[0,1,2,3,4,5,...|  -2.34695711077105|
|(10,[0,1,2,3,4,5,...|  5.411212414721781|
|(10,[0,1,2,3,4,5,...|0.27113285025860956|
|(10,[0,1,2,3,4,5,...|  3.592576452612441|
|(10,[0,1,2,3,4,5,...|  3.648156179616671|
|(10,[0,1,2,3,4,5,...|-2.0187490551333966|
|(10,[0,1,2,3,4,5,...|  4.666894740330598|
|(10,[0,1,2,3,4,5,...| 0.6339925075816553|
|(10,[0,1,2,3,4,5,...|   4.03577648414338|
|(10,[0,1,2,3,4,5,...| -4.212972857936261|
|(10,[0,1,2,3,4,5,...| -4.200823153983873|
|(10,[0,1,2,3,4,5,...| 3.5417001734228877|
|(10,[0,1,2,3,4,5,...|-1.0481946796515398|
|(10,[0,1,2,3,4,5,...|-0.5251375444396025|
+----------